## Trabajador experto en conocimiento

### Un agente que responde preguntas y es un trabajador experto en conocimiento
### Para ser utilizado por empleados de Insurellm, una empresa de tecnología de seguros
### El agente debe ser preciso y la solución debe ser de bajo costo.

Este proyecto utilizará RAG (Retrieval Augmented Generation, Generación aumentada de recuperación) para garantizar que nuestro asistente de preguntas y respuestas tenga una alta precisión.

Esta primera implementación utilizará un tipo de RAG simple, de fuerza bruta.

### Nota al margen: Aplicaciones comerciales de los proyectos de esta semana

¡RAG es quizás la técnica más inmediatamente aplicable de todo lo que cubrimos en el curso! De hecho, hay productos comerciales que hacen precisamente lo que construimos esta semana: consultas matizadas en grandes bases de datos de información, como contratos de la empresa o especificaciones de productos. RAG le brinda un mecanismo de bajo costo y rápida comercialización para adaptar un LLM a su área comercial.

In [1]:
# imports

import os
import glob
from dotenv import load_dotenv
import gradio as gr
from openai import OpenAI

In [2]:
# El precio es un factor para nuestra empresa, por eso vamos a utilizar un modelo de bajo costo.

MODEL = "gpt-4o-mini"

In [3]:
# Cargar variables de entorno en un archivo llamado .env

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
openai = OpenAI()

In [4]:
# ¡Un agradecimiento enorme al estudiante Dr. John S. por corregir un error a continuación para los usuarios de Windows!
context = {}

employees = glob.glob("knowledge-base/employees/*")

for employee in employees:
    name = employee.split(' ')[-1][:-3]
    doc = ""
    with open(employee, "r", encoding="utf-8") as f:
        doc = f.read()
    context[name]=doc

In [7]:
context["Lancaster"]

'# Avery Lancaster\n\n## Resumen\n- **Fecha de nacimiento**: 15 de marzo de 1985\n- **Puesto**: Cofundadora y directora ejecutiva (CEO)\n- **Ubicación**: San Francisco, California\n\n## Progresión profesional en Insurellm\n- **2015 - Presente**: Cofundadora y directora ejecutiva\nAvery Lancaster cofundó Insurellm en 2015 y desde entonces ha guiado a la empresa hasta su posición actual como proveedor líder de tecnología de seguros. Avery es conocida por sus innovadoras estrategias de liderazgo y su experiencia en gestión de riesgos que han catapultado a la empresa al mercado de seguros convencional.\n\n- **2013 - 2015**: Gerente sénior de productos en Innovate Insurance Solutions\nAntes de lanzar Insurellm, Avery fue gerente sénior de productos en Innovate Insurance Solutions, donde desarrolló productos de seguros innovadores destinados al sector tecnológico.\n\n- **2010 - 2013**: Analista de negocios en Edge Analytics\nAntes de unirse a Innovate, Avery trabajó como analista de negocios

In [8]:
products = glob.glob("knowledge-base/products/*")

for product in products:
    name = product.split(os.sep)[-1][:-3]
    doc = ""
    with open(product, "r", encoding="utf-8") as f:
        doc = f.read()
    context[name]=doc

In [9]:
context.keys()

dict_keys(['Chen', 'Spencer', 'Tran', 'Blake', 'Lancaster', 'Thompson', 'Greene', 'Thomson', 'Trenton', 'Harper', 'Bishop', 'Carter', 'Rellm', 'Markellm', 'Homellm', 'Carllm'])

In [11]:
system_message = "Eres un experto en responder preguntas precisas sobre Insurellm, la compañía de tecnología de seguros. Quiero que des respuestas breves y precisas. Si no sabes la respuesta, dilo. No inventes nada si no se te ha proporcionado el contexto relevante."

In [12]:
def get_relevant_context(message):
    relevant_context = []
    for context_title, context_details in context.items():
        if context_title.lower() in message.lower():
            relevant_context.append(context_details)
    return relevant_context          

In [15]:
get_relevant_context("¿Quién es Avery Lancaster?")

['# Avery Lancaster\n\n## Resumen\n- **Fecha de nacimiento**: 15 de marzo de 1985\n- **Puesto**: Cofundadora y directora ejecutiva (CEO)\n- **Ubicación**: San Francisco, California\n\n## Progresión profesional en Insurellm\n- **2015 - Presente**: Cofundadora y directora ejecutiva\nAvery Lancaster cofundó Insurellm en 2015 y desde entonces ha guiado a la empresa hasta su posición actual como proveedor líder de tecnología de seguros. Avery es conocida por sus innovadoras estrategias de liderazgo y su experiencia en gestión de riesgos que han catapultado a la empresa al mercado de seguros convencional.\n\n- **2013 - 2015**: Gerente sénior de productos en Innovate Insurance Solutions\nAntes de lanzar Insurellm, Avery fue gerente sénior de productos en Innovate Insurance Solutions, donde desarrolló productos de seguros innovadores destinados al sector tecnológico.\n\n- **2010 - 2013**: Analista de negocios en Edge Analytics\nAntes de unirse a Innovate, Avery trabajó como analista de negocio

In [18]:
get_relevant_context("¿Quién es Avery y qué es carllm?")

['# Resumen del producto\n\n# Carllm\n\n## Resumen\n\nCarllm es un innovador producto de seguros para automóviles desarrollado por Insurellm, diseñado para optimizar la forma en que las compañías de seguros ofrecen cobertura a sus clientes. Impulsado por inteligencia artificial de vanguardia, Carllm utiliza algoritmos avanzados para brindar soluciones de seguros para automóviles personalizadas, lo que garantiza una cobertura óptima y minimiza los costos. Con una infraestructura sólida que respalda a los clientes B2B y B2C, Carllm redefine el panorama de los seguros para automóviles y permite a los proveedores de seguros mejorar la satisfacción y la retención de los clientes.\n\n## Características\n\n- **Evaluación de riesgos impulsada por IA**: Carllm aprovecha la inteligencia artificial para analizar el comportamiento del conductor, las condiciones del vehículo y los datos históricos de reclamos. Esto permite a las aseguradoras tomar decisiones informadas y establecer primas competiti

In [19]:
def add_context(message):
    relevant_context = get_relevant_context(message)
    if relevant_context:
        message += "\n\El siguiente contexto adicional podría ser relevante para responder a esta pregunta:\n\n"
        for relevant in relevant_context:
            message += relevant + "\n\n"
    return message

In [21]:
print(add_context("¿Quién es Alex Lancaster?"))

¿Quién es Alex Lancaster?
\El siguiente contexto adicional podría ser relevante para responder a esta pregunta:

# Avery Lancaster

## Resumen
- **Fecha de nacimiento**: 15 de marzo de 1985
- **Puesto**: Cofundadora y directora ejecutiva (CEO)
- **Ubicación**: San Francisco, California

## Progresión profesional en Insurellm
- **2015 - Presente**: Cofundadora y directora ejecutiva
Avery Lancaster cofundó Insurellm en 2015 y desde entonces ha guiado a la empresa hasta su posición actual como proveedor líder de tecnología de seguros. Avery es conocida por sus innovadoras estrategias de liderazgo y su experiencia en gestión de riesgos que han catapultado a la empresa al mercado de seguros convencional.

- **2013 - 2015**: Gerente sénior de productos en Innovate Insurance Solutions
Antes de lanzar Insurellm, Avery fue gerente sénior de productos en Innovate Insurance Solutions, donde desarrolló productos de seguros innovadores destinados al sector tecnológico.

- **2010 - 2013**: Analista 

In [22]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history
    message = add_context(message)
    messages.append({"role": "user", "content": message})

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

## Ahora, lo mostraremos en Gradio usando la interfaz de Chat:

Una forma rápida y sencilla de crear un prototipo de chat con un LLM

In [25]:
view = gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.
